# SUUMO 和歌山市賃貸　ペルソナ絞り込みデータの加工

## 各物件に対する最寄りの公園データの算出

In [43]:
# import
import numpy as np
import pandas as pd
from geopy.distance import geodesic

In [44]:
# データ読み込み
df_home = pd.read_csv('20231009_SUUMO_和歌山市賃貸_ペルソナ絞り込み.csv', index_col=None)
df_park = pd.read_csv('いこーよ公園.csv', index_col=None)
df_lesson = pd.read_csv('20231004_習い事スクスク_スクレイピング_経度緯度追加.csv', index_col=None)
df_js = pd.read_csv('20231004_和歌山市小学校_スクレイピング_経度緯度追加.csv', index_col=None)
df_kg = pd.read_csv('20231004_和歌山市保育園_スクレイピング_経度緯度追加.csv', index_col=None)
df_kg2 = pd.read_csv('20231004_和歌山市幼稚園_スクレイピング_経度緯度追加.csv', index_col=None)

In [45]:
df_lesson.head()

,Club,Genre,Address,Access,Age,Feature,url,longitude,latitude
0,biima sports(ビーマ スポーツ) 和歌山校,サッカー・フットサル/野球・ソフトボール/バスケットボール/テニス/運動・体操,和歌山県和歌山市小雑賀239番地ワカヤマテニススクール内,きのくに線宮前駅,年少、年中、年⻑、⼩1、⼩2、⼩3、⼩4、⼩5、⼩6,土日あり/振替あり/駐車場あり,https://sp-sukusuku.jp/kyoushitsu/s22000000793,34.206296,135.182801
1,ラボ・パーティ 和歌山市松江教室(加地 亜希パーティ),英語・英会話/幼児教室,和歌山県和歌山市松江1478-44,南海本線和歌山市駅/南海本線紀ノ川駅,ベビー、年少、年中、年⻑、⼩1、⼩2、⼩3、⼩4、⼩5、⼩6,親子コースあり/駐車場あり,https://sp-sukusuku.jp/kyoushitsu/s19000010236,34.239794,135.154389
2,ECCジュニア 木ノ本教室,英語・英会話,和歌山県和歌山市木ノ本３３０－５,南海加太線八幡前駅/南海加太線中松江駅,ベビー、年少、年中、年⻑、⼩1、⼩2、⼩3、⼩4、⼩5、⼩6,NaN,https://sp-sukusuku.jp/kyoushitsu/s91200005937,34.258628,135.128129
3,ECCジュニア 神波教室,英語・英会話/学習塾,和歌山県和歌山市神波31-5,阪和線(天王寺～和歌山)紀伊駅,ベビー、年少、年中、年⻑、⼩1、⼩2、⼩3、⼩4、⼩5、⼩6,NaN,https://sp-sukusuku.jp/kyoushitsu/s91200005980,34.264392,135.255095
4,ラボ・パーティ 和歌山市中之島教室(加地 亜希パーティ),英語・英会話/幼児教室,和歌山県和歌山市中之島1288,阪和線(天王寺～和歌山)紀伊中ノ島駅/紀勢本線(和歌山～和歌山市)紀和駅/紀勢本線(和歌山～...,⼩1、⼩2、⼩3、⼩4、⼩5、⼩6,駅チカ/駐車場あり,https://sp-sukusuku.jp/kyoushitsu/s19000010341,34.240813,135.188322


In [46]:
df_js.head()

,School_Name,Stu_Number,Tea_Number,Class,Address,Access,url,longitude,latitude
0,芦原小学校,59人,17人,7学級/平均8.4人,和歌山市雄松町４－２５,宮前駅から912m,https://www.gaccom.jp/schools-22490.html,34.220365,135.181622
1,有功小学校,329人,22人,15学級/平均21.9人,和歌山市園部１４５３,六十谷駅から969m,https://www.gaccom.jp/schools-22513.html,34.264287,135.196363
2,有功東小学校,291人,22人,14学級/平均20.8人,和歌山市六十谷８９５,六十谷駅から671m,https://www.gaccom.jp/schools-22528.html,34.268419,135.204705
3,今福小学校,153人,15人,9学級/平均17.0人,和歌山市今福３-７-４６,和歌山港駅から1.8km,https://www.gaccom.jp/schools-22522.html,34.211140,135.164071
4,太田小学校,344人,21人,14学級/平均24.6人,和歌山市太田６３６,和歌山駅から697m,https://www.gaccom.jp/schools-22521.html,34.229335,135.197424


In [47]:
df_kg.head()

,Category,School,Address,url,longitude,latitude
0,市区町村,栄谷保育所,和歌山県和歌山市栄谷４２８－３,https://www.gaccom.jp/nurseryschool-18665.html,34.258560,135.161422
1,市区町村,本町こども園,和歌山県和歌山市北桶屋町７,https://www.gaccom.jp/nurseryschool-18670.html,34.235955,135.175611
2,市区町村,西和佐保育所,和歌山県和歌山市岩橋１３１７－４,https://www.gaccom.jp/nurseryschool-18671.html,34.232667,135.227178
3,市区町村,名草保育所,和歌山県和歌山市紀三井寺２８７,https://www.gaccom.jp/nurseryschool-18672.html,34.178529,135.190097
4,市区町村,宮北保育所,和歌山県和歌山市黒田３５４－１,https://www.gaccom.jp/nurseryschool-18676.html,34.238918,135.195467


In [48]:
df_kg2.head()

,Category,School,Address,url,longitude,latitude
0,公立,和歌山市立本町こども園,和歌山県和歌山市北桶屋町７,https://www.gaccom.jp/kindergarten-9781.html,34.235955,135.175611
1,公立,和歌山市立岡山幼稚園,和歌山県和歌山市吹上１丁目４－１,https://www.gaccom.jp/kindergarten-9782.html,34.222803,135.170821
2,公立,和歌山市立中之島幼稚園,和歌山県和歌山市中之島１４９５,https://www.gaccom.jp/kindergarten-9783.html,34.238850,135.186926
3,公立,和歌山市立芦原こども園,和歌山県和歌山市雄松町５－４－１,https://www.gaccom.jp/kindergarten-9784.html,34.219577,135.181028
4,公立,和歌山市立湊幼稚園,和歌山県和歌山市湊２－１７－４０,https://www.gaccom.jp/kindergarten-9785.html,34.233142,135.144672


In [49]:
def find_nearest_park(property_lat, property_lon, df_park):
    nearest_park = None
    min_distance = float('inf')  # 初期距離を無限大に設定

    for index, row in df_park.iterrows():
        park_lat = row['緯度']
        park_lon = row['経度']
        distance = geodesic((property_lat, property_lon), (park_lat, park_lon)).kilometers

        if distance < min_distance:
            min_distance = distance
            nearest_park = row['title']

    return nearest_park, min_distance

In [50]:
# 物件から最も近い病院を計算し、結果を新しい列に追加
df_home['最寄りの公園'], df_home['最寄りの公園_距離(km)'] = zip(*df_home.apply(
    lambda row: find_nearest_park(row['longitude'], row['latitude'], df_park),
    axis=1
))

In [51]:
# カラム名に「公園_」を追記
df_park.columns = ['公園_' + col for col in df_park.columns]
df_park.head()

,公園_title,公園_address,公園_facility,公園_緯度,公園_経度
0,アドベンチャーワールド,和歌山県西牟婁郡白浜町堅田2399,８０万平方メートルの広大な敷地を持ち、動物園や遊園地、マリンワールドなどバラエティーが豊富な...,33.677686,135.377419
1,海南nobinos（ノビノス）,和歌山県海南市日方1525-6,子どもが騒いでも安心な、“遊べる”公共図書館です。\r\n日本一の開架冊数となる5万冊の絵本...,34.155273,135.209233
2,和歌山県植物公園緑花センター,和歌山県岩出市東坂本672,1979年に和歌山県岩出市の郊外にオープンした、花や緑が豊かな公園です。四季の花々を植えてい...,34.256226,135.311339
3,平草原公園,和歌山県西牟婁郡白浜町2054-1,和歌山県西牟婁郡の白浜町に位置し、西に紀伊水道、南に熊野枯木灘県立公園、東に熊野三千六百峰の...,33.678140,135.348190
4,東松江緑地,和歌山県和歌山市松江字29町場2000,河西緩衝緑地の一番東側にある公園です。\r\nキャッチボールしたり寝転がって楽しめる広い芝生...,34.238586,135.128405


In [52]:
# "最寄りの公園"列と"公園_title"列をキーとして横方向に結合
df_park_merge = pd.merge(df_home, df_park, left_on='最寄りの公園', right_on='公園_title', how='inner')
# 不要なキー列を削除
df_park_merge.drop('公園_title', axis=1, inplace=True)

In [54]:
def find_nearest_kg(property_lat, property_lon, df_kg):
    nearest_kg = None
    min_distance = float('inf')  # 初期距離を無限大に設定

    for index, row in df_kg.iterrows():
        kg_lat = row['longitude']
        kg_lon = row['latitude']
        distance = geodesic((property_lat, property_lon), (kg_lat, kg_lon)).kilometers

        if distance < min_distance:
            min_distance = distance
            nearest_kg = row['School']

    return nearest_kg, min_distance


In [55]:
# 物件から最も近い保育園を計算し、結果を新しい列に追加
df_park_merge['最寄りの保育園'], df_park_merge['最寄りの保育園_距離(km)'] = zip(*df_park_merge.apply(
    lambda row: find_nearest_kg(row['longitude'], row['latitude'], df_kg),
    axis=1
))

In [56]:
# カラム名に「保育園_」を追記
df_kg.columns = ['保育園_' + col for col in df_kg.columns]

In [57]:
# "最寄りの保育園"列と"保育園_School"列をキーとして横方向に結合
df_kg_merge = pd.merge(df_park_merge, df_kg, left_on='最寄りの保育園', right_on='保育園_School', how='inner')
# 不要なキー列を削除
df_kg_merge.drop('保育園_School', axis=1, inplace=True)

In [58]:
def find_nearest_kg2(property_lat, property_lon, df_kg2):
    nearest_kg2 = None
    min_distance = float('inf')  # 初期距離を無限大に設定

    for index, row in df_kg2.iterrows():
        kg2_lat = row['longitude']
        kg2_lon = row['latitude']
        distance = geodesic((property_lat, property_lon), (kg2_lat, kg2_lon)).kilometers

        if distance < min_distance:
            min_distance = distance
            nearest_kg2 = row['School']

    return nearest_kg2, min_distance

In [59]:
# 物件から最も近い幼稚園を計算し、結果を新しい列に追加
df_kg_merge['最寄りの幼稚園'], df_kg_merge['最寄りの幼稚園_距離(km)'] = zip(*df_kg_merge.apply(
    lambda row: find_nearest_kg2(row['longitude'], row['latitude'], df_kg2),
    axis=1
))

In [60]:
# カラム名に「幼稚園_」を追記
df_kg2.columns = ['幼稚園_' + col for col in df_kg2.columns]

In [61]:
# "最寄りの幼稚園"列と"幼稚園_School"列をキーとして横方向に結合
df_kg2_merge = pd.merge(df_kg_merge, df_kg2, left_on='最寄りの幼稚園', right_on='幼稚園_School', how='inner')
# 不要なキー列を削除
df_kg2_merge.drop('幼稚園_School', axis=1, inplace=True)

In [62]:
def find_nearest_js(property_lat, property_lon, df_js):
    nearest_js = None
    min_distance = float('inf')  # 初期距離を無限大に設定

    for index, row in df_js.iterrows():
        js_lat = row['longitude']
        js_lon = row['latitude']
        distance = geodesic((property_lat, property_lon), (js_lat, js_lon)).kilometers

        if distance < min_distance:
            min_distance = distance
            nearest_js = row['School_Name']

    return nearest_js, min_distance


In [63]:
# 物件から最も近い小学校を計算し、結果を新しい列に追加
df_kg2_merge['最寄りの小学校'], df_kg2_merge['最寄りの小学校_距離(km)'] = zip(*df_kg2_merge.apply(
    lambda row: find_nearest_js(row['longitude'], row['latitude'], df_js),
    axis=1
))

In [64]:
# カラム名に「小学校_」を追記
df_js.columns = ['小学校_' + col for col in df_js.columns]

In [65]:
# "最寄りの小学校"列と"小学校_School_Name"列をキーとして横方向に結合
df_js_merge = pd.merge(df_kg2_merge, df_js, left_on='最寄りの小学校', right_on='小学校_School_Name', how='inner')
# 不要なキー列を削除
df_js_merge.drop('小学校_School_Name', axis=1, inplace=True)

In [66]:
pd.set_option("display.max_columns", None)
df_js_merge.head(1)

,カテゴリ,物件名,住所,最寄りのアクセス_1,最寄りのアクセス_2,最寄りのアクセス_3,築年数_年,階数,階,家賃_万円,管理費_万円,敷金_万円,礼金_万円,間取,面積_m2,URL,longitude,latitude,Address,Area,Group,Block,最寄りの公園,最寄りの公園_距離(km),公園_address,公園_facility,公園_緯度,公園_経度,最寄りの保育園,最寄りの保育園_距離(km),保育園_Category,保育園_Address,保育園_url,保育園_longitude,保育園_latitude,最寄りの幼稚園,最寄りの幼稚園_距離(km),幼稚園_Category,幼稚園_Address,幼稚園_url,幼稚園_longitude,幼稚園_latitude,最寄りの小学校,最寄りの小学校_距離(km),小学校_Stu_Number,小学校_Tea_Number,小学校_Class,小学校_Address,小学校_Access,小学校_url,小学校_longitude,小学校_latitude
0,賃貸一戸建て,It`s毛見,和歌山県和歌山市毛見,ＪＲ紀勢本線/黒江駅 歩34分,ＪＲ紀勢本線/紀三井寺駅 歩38分,ＪＲ紀勢本線/海南駅 歩43分,0,2階建,1-2階,12.0,0.0,12.0,24.0,3LDK,97.7,https://suumo.jp/chintai/jnc_000083770016/?bc=...,34.166366,135.187311,和歌山県和歌山市毛見,名草地区,第7ブロック,南部地域,紀三井寺公園,0.43224,和歌山県和歌山市毛見200番地,"「紀三井寺公園」は面積約172,300平方メートル内に、陸上競技場、野球場、テニス場、球技場...",34.167907,135.191617,名草幼稚園（保）,1.03059,その他の法人,和歌山県和歌山市紀三井寺５２０,https://www.gaccom.jp/nurseryschool-91752.html,34.175481,135.189477,名草幼稚園（幼）,1.03059,私立,和歌山県和歌山市紀三井寺５２０,https://www.gaccom.jp/kindergarten-9886.html,34.175481,135.189477,浜宮小学校,0.94402,373人,23人,16学級/平均23.3人,和歌山市内原７７８-１６,黒江駅から1.3km,https://www.gaccom.jp/schools-22499.html,34.170456,135.19629


In [67]:
df_js_merge.to_csv('20231010_SUUMO_和歌山市賃貸_最寄り施設_詳細.csv', index=None)